Script for stratifying domestic versus international flights for business air travel carbon mitigation program at UC Berkeley
Updated: October 16th, 2025
Author: Ashley Smiley; ashley.smiley@berkeley.edu

Booking metrics needed to run this script, along with departmental codes and fund keys
Those data uploaded separately

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**TNE Flights**

In [21]:
TNEFlights2023 = pd.read_csv('/Users/ashley/TNE2023.csv')

In [22]:
len(TNEFlights2023['Invoice ID (or DBID)'])

9899

In [23]:
TNEFlights2023['Invoice ID (or DBID)'].nunique()

9494

In [24]:
TNEFlights2023.shape

(9899, 17)

In [25]:
TNEFlights2023.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
ChartField 1             object
ChartField 2             object
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [26]:
TNEFlights2023.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,ChartField 1,ChartField 2,Payment Method,Account Descr,Amount
0,TRV,90499066,"BARANI,MOHAMMAD",1/1/23,TRV000609648,1/13/23,3134.51,TRV000609648;BARANI,3134.51,57212,61939,24532,NaN,JBMAF,EFT,Out-of-State Travel - Air Fare,719.30
1,TRV,90498399,"HARRIS,EVA",1/1/23,TRV000609649,1/9/23,1980.81,TRV000609649;HARRIS,1980.81,57221,62558,14000,NaN,CPH51,EFT,Foreign Travel - Air Fare,882.81
2,TRV,90510103,"COHEN,RONALD",1/2/23,TRV000609654,4/7/23,2605.26,TRV000609654;COHEN,2605.26,57212,66350,10805,NaN,CCRXC,EFT,Out-of-State Travel - Air Fare,419.95
3,TRV,90498726,"LIUZZO,LUCAS",1/2/23,TRV000609657,1/11/23,2594.20,TRV000609657;LIUZZO,2594.20,57212,61043,24532,NaN,JBANP,EFT,Out-of-State Travel - Air Fare,603.05
4,TRV,90499104,"BAMMAN,DAVID",1/3/23,TRV000609694,1/13/23,3761.30,136371TRV000609694;BAMMAN,3761.30,57221,19900,14208,NaN,MMDAB,EFT,Foreign Travel - Air Fare,1635.67


In [27]:
TNEFlights2023['Account'].value_counts()

Account
57212    4115
57211    3032
57221    2540
56715     212
Name: count, dtype: int64

In [29]:
TNEFlights2023['Line Description'].head(10)

0          TRV000609648;BARANI
1          TRV000609649;HARRIS
2           TRV000609654;COHEN
3          TRV000609657;LIUZZO
4    136371TRV000609694;BAMMAN
5            TRV000609702;CHOI
6          TRV000609672;CURTIS
7         136309ACollinsSUPERB
8                    ID 136470
9                    ID 136471
Name: Line Description, dtype: object

In [8]:
drop_chartfields = ['ChartField 1', 'ChartField 2']

In [9]:
TNEFlights2023.drop(drop_chartfields, axis=1, inplace=True)

In [10]:
TNEFlights2023.shape

(9899, 15)

**BCD Flights**

In [30]:
BCD2023 = pd.read_csv('/Users/ashley/BCD2023.csv')

In [12]:
BCD2023['Invoice ID (or DBID)'] = BCD2023['Invoice ID (or DBID)'].map(lambda x: str(x)[:-2])

In [31]:
BCD2023['Invoice ID (or DBID)'].nunique()

2155

In [32]:
BCD2023.shape

(3907, 17)

In [15]:
AirFareFees = [9.75, 34.00]
BCD2023[BCD2023.Amount.isin(AirFareFees) == False]
BCD2023minusAirFareFees = BCD2023[BCD2023.Amount.isin(AirFareFees) == False]
BCDFlights2023 = BCD2023minusAirFareFees[BCD2023minusAirFareFees['Amount'] > 0]

In [33]:
BCDFlights2023.shape

(2006, 17)

In [34]:
BCDFlights2023['Account'].value_counts()

Account
57239    1702
57211     140
57212     112
57221      41
56715      11
Name: count, dtype: int64

In [35]:
BCDFlights2023.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
ChartField 1             object
ChartField 2             object
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [13]:
BCDFlights2023.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,ChartField 1,ChartField 2,Payment Method,Account Descr,Amount
0,TRV,13632554,US BANK BCD TRAVEL,1/2/23,5800308312,1/10/23,25.00,No Description,25.00,57239,69995,750,NaN,NaN,EFT,Travel DB US Bank Clearing,25.00
1,TRV,13632822,US BANK BCD TRAVEL,1/3/23,5800510702,1/11/23,297.80,No Description,297.80,57239,69995,750,NaN,NaN,EFT,Travel DB US Bank Clearing,297.80
2,TRV,13632819,US BANK BCD TRAVEL,1/3/23,5800510762,1/11/23,301.49,No Description,301.49,57239,69995,750,NaN,NaN,EFT,Travel DB US Bank Clearing,301.49
5,TRV,13633098,US BANK BCD TRAVEL,1/4/23,5800610989,1/12/23,220.27,No Description,220.27,57239,69995,750,NaN,NaN,EFT,Travel DB US Bank Clearing,220.27
6,TRV,13633099,US BANK BCD TRAVEL,1/4/23,5800613128,1/12/23,473.58,No Description,473.58,57239,69995,750,NaN,NaN,EFT,Travel DB US Bank Clearing,473.58


In [18]:
BCDFlights2023 = BCDFlights2023[BCDFlights2023['Account'] != 57239]

In [19]:
BCDFlights2023.shape

(304, 17)

In [20]:
BCDFlights2023.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
ChartField 1             object
ChartField 2             object
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

**Load data Laura shared that clears up some of the 57239 accounts**

**I start working with the new data here**

In [21]:
BCDnewstack = pd.read_csv('/Users/ashley/BCD2023update.csv')

In [22]:
BCDnewstack.shape

(3243, 23)

In [23]:
BCDnewstack.drop('Function', axis=1, inplace=True)

In [24]:
more_columns_to_delete = ['Ticket Number', 'BFS ticket # after removed Airline codes', 'TicketNbr']

In [25]:
BCDnewstack.drop(more_columns_to_delete, axis=1, inplace=True)

In [26]:
BCDnewstack.shape

(3243, 19)

**Replace the "missing" DBIDs with their correct ones**

In [27]:
BCDnewstack.iloc[:, 4] = BCDnewstack.iloc[:, -2]

In [28]:
BCDnewstack.drop('Direct Bill ID-S5', axis=1, inplace=True)

In [29]:
BCDnewstack.shape

(3243, 18)

**Laura gave us 4+ columns of data in one cell, so I broke it out here**

In [30]:
BCDnewstack['COA'] = BCDnewstack['COA'].str.split('-').str[:4]

In [31]:
BCDnewstack[['delete', 'AccountCorrection', 'FundCorrection', 'DeptidCorrection']] = pd.DataFrame(BCDnewstack['COA'].tolist())

In [32]:
BCDnewstack.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,...,ChartField 1,ChartField 2,Payment Method,Account Descr,Amount,COA,delete,AccountCorrection,FundCorrection,DeptidCorrection
0,TRV,13649777,US BANK BCD TRAVEL,2/23/23,ABDE094372,3/7/23,462.67,No Description,462.67,57239,...,NaN,NaN,EFT,Travel DB US Bank Clearing,462.67,"[1, 57212, 20225, 11981]",1,57212,20225,11981
1,TRV,13649802,US BANK BCD TRAVEL,2/23/23,ABDE094372,3/7/23,9.75,No Description,9.75,57239,...,NaN,NaN,EFT,Travel DB US Bank Clearing,9.75,"[1, 57212, 20225, 11981]",1,57212,20225,11981
2,TRV,13659510,US BANK BCD TRAVEL,3/17/23,ACRE095756,4/4/23,775.12,No Description,775.12,57239,...,NaN,NaN,EFT,Travel DB US Bank Clearing,775.12,"[1, 57211, 96576, 31178]",1,57211,96576,31178
3,TRV,13659522,US BANK BCD TRAVEL,3/17/23,ACRE095756,4/4/23,9.75,No Description,9.75,57239,...,NaN,NaN,EFT,Travel DB US Bank Clearing,9.75,"[1, 57211, 96576, 31178]",1,57211,96576,31178
4,TRV,13701980,US BANK BCD TRAVEL,9/5/23,ADEL101812,9/12/23,298.97,No Description,298.97,57239,...,NaN,NaN,EFT,Travel DB US Bank Clearing,298.97,"[1, 57211, 70450, 23398]",1,57211,70450,23398


In [33]:
BCDnewstack.drop('delete', axis=1, inplace=True)

**I replaced the 57239 accounts with their correct ones as well as the Fund & Deptid codes**

In [34]:
BCDnewstack['Account'] = BCDnewstack['AccountCorrection']

In [35]:
BCDnewstack['Fund'] = BCDnewstack['FundCorrection']

In [36]:
BCDnewstack['Deptid'] = BCDnewstack['DeptidCorrection']

In [37]:
BCDnewstack.shape

(3243, 21)

In [38]:
column_cleanup = ['AccountCorrection', 'FundCorrection', 'DeptidCorrection', 'COA']

In [39]:
BCDnewstack.drop(column_cleanup, axis=1, inplace=True)

In [40]:
BCDnewstack.shape

(3243, 17)

In [41]:
BCDnewstack.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                  object
Fund                     object
Deptid                   object
ChartField 1            float64
ChartField 2            float64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

**Made sure all columns in both dataframes have matching datatypes before stacking**

In [42]:
BCDnewstack['Account'] = BCDnewstack['Account'].astype(int)

In [43]:
BCDnewstack['Fund'] = BCDnewstack['Fund'].astype(int)

In [44]:
BCDnewstack['Deptid'] = BCDnewstack['Deptid'].astype(int)

In [45]:
BCDnewstack.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,ChartField 1,ChartField 2,Payment Method,Account Descr,Amount
0,TRV,13649777,US BANK BCD TRAVEL,2/23/23,ABDE094372,3/7/23,462.67,No Description,462.67,57212,20225,11981,NaN,NaN,EFT,Travel DB US Bank Clearing,462.67
1,TRV,13649802,US BANK BCD TRAVEL,2/23/23,ABDE094372,3/7/23,9.75,No Description,9.75,57212,20225,11981,NaN,NaN,EFT,Travel DB US Bank Clearing,9.75
2,TRV,13659510,US BANK BCD TRAVEL,3/17/23,ACRE095756,4/4/23,775.12,No Description,775.12,57211,96576,31178,NaN,NaN,EFT,Travel DB US Bank Clearing,775.12
3,TRV,13659522,US BANK BCD TRAVEL,3/17/23,ACRE095756,4/4/23,9.75,No Description,9.75,57211,96576,31178,NaN,NaN,EFT,Travel DB US Bank Clearing,9.75
4,TRV,13701980,US BANK BCD TRAVEL,9/5/23,ADEL101812,9/12/23,298.97,No Description,298.97,57211,70450,23398,NaN,NaN,EFT,Travel DB US Bank Clearing,298.97


In [46]:
BCDnewstack.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
ChartField 1            float64
ChartField 2            float64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

**Subtracted airfare fees from new dataset and removed negative amounts**

In [47]:
AirFareFees = [9.75, 34.00]
BCDnewstack[BCDnewstack.Amount.isin(AirFareFees) == False]
BCDnewstackminusAirFareFees = BCDnewstack[BCDnewstack.Amount.isin(AirFareFees) == False]
BCDnewstackFlights = BCDnewstackminusAirFareFees[BCDnewstackminusAirFareFees['Amount'] > 0]

In [48]:
BCDnewstackFlights.shape

(1644, 17)

In [49]:
BCDnewstackFlights['Invoice ID (or DBID)'].nunique()

1270

In [50]:
BCDnewstackFlights.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
ChartField 1            float64
ChartField 2            float64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

**These column datatypes were causing problems, we don't need them so i removed them**

In [51]:
drop_chartfields = ['ChartField 1', 'ChartField 2']

In [52]:
BCDFlights2023.drop(drop_chartfields, axis=1, inplace=True)

In [55]:
BCDnewstackFlights.drop(drop_chartfields, axis=1, inplace=True)

KeyError: "['ChartField 1', 'ChartField 2'] not found in axis"

In [57]:
BCDnewstackFlights.shape

(1644, 15)

In [58]:
BCDFlights2023.shape

(304, 15)

**Combine new data with original data - no duplicates**

In [59]:
BCDFlights2023 = pd.concat([BCDFlights2023, BCDnewstackFlights], ignore_index=True)

In [60]:
BCDFlights2023.shape

(1948, 15)

In [61]:
BCDFlights2023.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,Payment Method,Account Descr,Amount
0,TRV,13634928,US BANK BCD TRAVEL,1/5/23,5801111229,1/19/23,92.83,ADEL092834 SFO-TUS-SFO,92.83,57212,70450,750,EFT,Out-of-State Travel - Air Fare,92.83
1,TRV,13634666,US BANK BCD TRAVEL,1/5/23,DAVA093010,1/18/23,400.46,GOOD092760 JFK-SFO-DFW-CUN,400.46,57212,19942,17819,EFT,Out-of-State Travel - Air Fare,400.46
2,TRV,13634670,US BANK BCD TRAVEL,1/5/23,QUES093019,1/18/23,848.80,MOHA092852 SFO-DCA-SFO,848.80,57212,19942,10935,EFT,Out-of-State Travel - Air Fare,848.80
3,TRV,13634680,US BANK BCD TRAVEL,1/5/23,AGUI092999,1/18/23,632.45,PINK092828 LHR-SFO-LHR,632.45,57221,44262,13250,EFT,Foreign Travel - Air Fare,632.45
4,TRV,13634658,US BANK BCD TRAVEL,1/6/23,RAY0093058,1/18/23,2025.75,ANSE092838 LHR-SFO-LHR-XXX-XXX,2025.75,57221,37797,13121,EFT,Foreign Travel - Air Fare,2025.75


**made sure all 57239 codes are removed (i.e., corrected)**

In [62]:
BCDFlights2023['Account'].value_counts()

Account
57211    897
57212    712
57221    262
56715     55
57232     13
57213      4
57222      3
55030      1
57214      1
Name: count, dtype: int64

In [63]:
BCDFlights2023.shape

(1948, 15)

In [64]:
BCDFlights2023['Invoice ID (or DBID)'].nunique()

1508

**UCTC Flights**

In [36]:
UCTC2023 = pd.read_csv('/Users/ashley/UCTC2023.csv')

In [37]:
UCTC2023['Invoice ID (or DBID)'] = UCTC2023['Invoice ID (or DBID)'].map(lambda x: str(x)[:-2])

In [38]:
AirFareFeesUCTC2023 = [10, 32]
UCTC2023[UCTC2023.Amount.isin(AirFareFeesUCTC2023) == False]
UCTC2023minusAirFareFees = UCTC2023[UCTC2023.Amount.isin(AirFareFeesUCTC2023) == False]
UCTCFlights2023 = UCTC2023minusAirFareFees[UCTC2023minusAirFareFees['Amount'] > 0]

In [39]:
UCTCFlights2023.shape

(10013, 17)

In [40]:
UCTCFlights2023['Invoice ID (or DBID)'].nunique()

6578

In [41]:
UCTCFlights2023['Account'].value_counts()

Account
57239    5168
57212    2605
57221    1260
57211     910
56715      70
Name: count, dtype: int64

In [42]:
UCTCFlights2023 = UCTCFlights2023[UCTCFlights2023['Account'] != 57239]

In [43]:
UCTCFlights2023.shape

(4845, 17)

In [44]:
UCTCFlights2023['Invoice ID (or DBID)'].nunique()

4031

In [45]:
drop_chartfields = ['ChartField 1', 'ChartField 2']
UCTCFlights2023.drop(drop_chartfields, axis=1, inplace=True)

In [46]:
UCTCFlights2023.shape

(4845, 15)

In [47]:
UCTCFlights2023.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [48]:
UCTCnewstack = pd.read_csv('/Users/ashley/UCTC2023update.csv')

In [49]:
UCTCnewstack.shape

(6738, 17)

In [50]:
UCTCnewstack.dtypes

Origin                      object
Voucher ID                   int64
Vendor Name                 object
Invoice Date                object
Invoice ID (or DBID)        object
Payment Date                object
Invoice Amt                float64
Line Description            object
Paid Gross Amount          float64
Account                      int64
Fund                         int64
Deptid                       int64
Payment Method              object
Account Descr               object
Amount                     float64
DBIDS from Agency Files     object
COA                         object
dtype: object

In [51]:
UCTCnewstack.iloc[:, 4] = UCTCnewstack.iloc[:, -2]

In [193]:
UCTCnewstack.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,Payment Method,Account Descr,Amount,DBIDS from Agency Files,COA
0,TRV,13633064,US BANK UC TRAVEL CENTER,1/4/23,HOLL092723,1/12/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,HOLL092723,1-19780-01489-15240- -12788A-
1,TRV,13632810,US BANK UC TRAVEL CENTER,1/3/23,ORD0092614,1/11/23,32.0,No Description,32.0,57239,69995,750,EFT,Travel DB US Bank Clearing,32.0,ORD0092614,1-57211-19900-13600-40- -MESRC
2,TRV,13638550,US BANK UC TRAVEL CENTER,1/24/23,LIZA092305,2/1/23,32.0,No Description,32.0,57239,69995,750,EFT,Travel DB US Bank Clearing,32.0,LIZA092305,1-57211-19924-11389-68- -
3,TRV,13719076,US BANK UC TRAVEL CENTER,10/30/23,JOHN090657,11/7/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,JOHN090657,1-57211-20136-31612-40-OMMKTG-OMBLM
4,TRV,13719082,US BANK UC TRAVEL CENTER,10/30/23,JOHN090657,11/7/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,JOHN090657,1-57211-20136-31612-40-OMMKTG-OMBLM


In [52]:
UCTCnewstack.drop('DBIDS from Agency Files', axis=1, inplace=True)

In [53]:
UCTCnewstack['COA'] = UCTCnewstack['COA'].str.split('-').str[:4]

In [54]:
UCTCnewstack[['delete', 'AccountCorrection', 'FundCorrection', 'DeptidCorrection']] = pd.DataFrame(UCTCnewstack['COA'].tolist())

In [55]:
UCTCnewstack.head(5)

,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,Payment Method,Account Descr,Amount,COA,delete,AccountCorrection,FundCorrection,DeptidCorrection
0,TRV,13633064,US BANK UC TRAVEL CENTER,1/4/23,HOLL092723,1/12/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,"[1, 19780, 01489, 15240]",1,19780,01489,15240
1,TRV,13632810,US BANK UC TRAVEL CENTER,1/3/23,ORD0092614,1/11/23,32.0,No Description,32.0,57239,69995,750,EFT,Travel DB US Bank Clearing,32.0,"[1, 57211, 19900, 13600]",1,57211,19900,13600
2,TRV,13638550,US BANK UC TRAVEL CENTER,1/24/23,LIZA092305,2/1/23,32.0,No Description,32.0,57239,69995,750,EFT,Travel DB US Bank Clearing,32.0,"[1, 57211, 19924, 11389]",1,57211,19924,11389
3,TRV,13719076,US BANK UC TRAVEL CENTER,10/30/23,JOHN090657,11/7/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,"[1, 57211, 20136, 31612]",1,57211,20136,31612
4,TRV,13719082,US BANK UC TRAVEL CENTER,10/30/23,JOHN090657,11/7/23,10.0,No Description,10.0,57239,69995,750,EFT,Travel DB US Bank Clearing,10.0,"[1, 57211, 20136, 31612]",1,57211,20136,31612


In [56]:
UCTCnewstack['Account'] = UCTCnewstack['AccountCorrection']
UCTCnewstack['Fund'] = UCTCnewstack['FundCorrection']
UCTCnewstack['Deptid'] = UCTCnewstack['DeptidCorrection']

In [57]:
column_cleanup = ['AccountCorrection', 'FundCorrection', 'DeptidCorrection', 'COA']
UCTCnewstack.drop(column_cleanup, axis=1, inplace=True)

In [58]:
UCTCnewstack.drop('delete', axis=1, inplace=True)

In [59]:
UCTCnewstack.shape

(6738, 15)

In [60]:
UCTCnewstack.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                  object
Fund                     object
Deptid                   object
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [61]:
UCTCnewstack['Account'] = UCTCnewstack['Account'].astype(int)
UCTCnewstack['Fund'] = UCTCnewstack['Fund'].astype(int)
UCTCnewstack['Deptid'] = UCTCnewstack['Deptid'].astype(int)

In [62]:
UCTCnewstack.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [63]:
AirFareFeesUCTC2023 = [10, 32]
UCTCnewstack[UCTCnewstack.Amount.isin(AirFareFeesUCTC2023) == False]
UCTCnewstackminusAirFareFees = UCTCnewstack[UCTCnewstack.Amount.isin(AirFareFeesUCTC2023) == False]
UCTCnewstackFlights = UCTCnewstackminusAirFareFees[UCTCnewstackminusAirFareFees['Amount'] > 0]

In [64]:
UCTCFlights2023.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [65]:
UCTCFlights2023.shape

(4845, 15)

In [66]:
UCTCFlights2023['Line Description'].head(10)

63                 AGGA092281 LAX-OAK
64     BUSH092565 SFO-LHR-DEL-CDG-SFO
65             MOOR092758 SFO-ORD-SFO
66     CEOL092522 TLS-CDG-SFO-CDG-TLS
138            WAGN092841 SFO-TLV-SFO
145    NIEL092839 CLE-CLT-SFO-DFW-CLE
146    REIN092848 SFO-EWR-GRU-ORD-SFO
148            BALZ092843 BDL-ORD-SEA
150            BALZ092845 SFO-ORD-BDL
153    BLAC092835 SYR-DTW-SFO-DTW-SYR
Name: Line Description, dtype: object

In [97]:
UCTCFlights2023['Invoice ID (or DBID)'].nunique()

4031

In [98]:
UCTCnewstackFlights.dtypes

Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [99]:
UCTCnewstackFlights.shape

(1611, 15)

In [100]:
UCTCnewstackFlights['Invoice ID (or DBID)'].nunique()

1403

**Stack 2 dataframes**

In [101]:
UCTCFlights2023 = pd.concat([UCTCFlights2023, UCTCnewstackFlights], ignore_index=True)

In [102]:
UCTCFlights2023.shape

(6456, 15)

In [103]:
UCTCFlights2023['Invoice ID (or DBID)'].nunique()

5322

In [104]:
UCTCFlights2023['Account'].value_counts()

Account
57212    3462
57221    1662
57211    1213
56715      83
57214      11
57222      10
55040       6
57003       3
57213       3
56520       1
57005       1
57232       1
Name: count, dtype: int64

**Insert a column for each agency so that we can map its origin after we stack all 3 DataFrames vertically**

In [105]:
BCDFlights2023.insert(0, 'which_agency', 'BCD')
UCTCFlights2023.insert(0, 'which_agency', 'UCTC')

In [106]:
TNEFlights2023.insert(0, 'which_agency', 'TNE')

In [107]:
TNEFlights2023.shape

(9899, 16)

In [108]:
BCDFlights2023.shape

(1948, 16)

In [109]:
BCDFlights2023.dtypes

which_agency             object
Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [110]:
UCTCFlights2023.shape

(6456, 16)

In [112]:
UCTCFlights2023.dtypes

which_agency             object
Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

In [113]:
TNEFlights2023.dtypes

which_agency             object
Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
dtype: object

**Stack the 3 DataFrames vertically into one TotalFlights2023 DataFrame**

**NOTE - columns must match EXACTLY in each dataframe for this step to work correctly. Otherwise, dataframes will stack horizontally**

In [114]:
TotalFlights2023 = pd.concat([TNEFlights2023, BCDFlights2023, UCTCFlights2023], axis=0, ignore_index=True)

In [115]:
TotalFlights2023.shape

(18303, 16)

In [65]:
TotalFlights2023.head()

,which_agency,Origin,Voucher ID,Vendor Name,Invoice Date,Invoice ID (or DBID),Payment Date,Invoice Amt,Line Description,Paid Gross Amount,Account,Fund,Deptid,ChartField 1,ChartField 2,Payment Method,Account Descr,Amount
0,TNE,TRV,90499066,"BARANI,MOHAMMAD",1/1/23,TRV000609648,1/13/23,3134.51,TRV000609648;BARANI,3134.51,57212,61939,24532,NaN,JBMAF,EFT,Out-of-State Travel - Air Fare,719.3
1,TNE,TRV,90498399,"HARRIS,EVA",1/1/23,TRV000609649,1/9/23,1980.81,TRV000609649;HARRIS,1980.81,57221,62558,14000,NaN,CPH51,EFT,Foreign Travel - Air Fare,882.81
2,TNE,TRV,90510103,"COHEN,RONALD",1/2/23,TRV000609654,4/7/23,2605.26,TRV000609654;COHEN,2605.26,57212,66350,10805,NaN,CCRXC,EFT,Out-of-State Travel - Air Fare,419.95
3,TNE,TRV,90498726,"LIUZZO,LUCAS",1/2/23,TRV000609657,1/11/23,2594.20,TRV000609657;LIUZZO,2594.20,57212,61043,24532,NaN,JBANP,EFT,Out-of-State Travel - Air Fare,603.05
4,TNE,TRV,90499104,"BAMMAN,DAVID",1/3/23,TRV000609694,1/13/23,3761.30,136371TRV000609694;BAMMAN,3761.30,57221,19900,14208,NaN,MMDAB,EFT,Foreign Travel - Air Fare,"1,635.67"


In [116]:
TotalFlights2023['Account'].nunique()

13

In [117]:
TotalFlights2023['Account'].value_counts()

Account
57212    8289
57211    5142
57221    4464
56715     350
57232      14
57222      13
57214      12
57213       7
55040       6
57003       3
55030       1
56520       1
57005       1
Name: count, dtype: int64

In [118]:
def ATMF(value):
    
    if value == 57211:
        return 'domestic'
    if value == 57212:
        return 'domestic'
    if value == 57221:
        return 'international'
    if value == 56715:
        return 'compedVisitor'
    if value == 57239:
        return 'stagingArea'
    
    else:
        return 'unknown'

In [119]:
TotalFlights2023['ATMF'] = TotalFlights2023['Account'].map(ATMF)

In [120]:
TotalFlights2023['ATMF'].nunique()

4

In [121]:
TotalFlights2023['ATMF'].value_counts()

ATMF
domestic         13431
international     4464
compedVisitor      350
unknown             58
Name: count, dtype: int64

**Add in a Restriction Type column**

In [122]:
fund_key_updated = pd.read_csv('/Users/ashley/fund_key_updated.csv')

In [123]:
TotalFlights2023['Restriction'] = TotalFlights2023['Fund'].map(fund_key_updated.set_index('Fund')['Restriction'])

In [124]:
TotalFlights2023['Restriction'].nunique()

5

In [125]:
TotalFlights2023['Restriction'].value_counts()

Restriction
R    10244
U     5671
G     1257
D      825
C      245
Name: count, dtype: int64

In [126]:
def RestrictionUpdate(series):
    
    if series == 'R':
        return 'R'
    if series == 'C':
        return 'R'
    if series == 'D':
        return 'R'
    if series == 'G':
        return 'R'
    if series == 'U':
        return 'U'
    else:
        return 'missingdata'

In [135]:
TotalFlights2023['RestrictionUpdate'] = TotalFlights2023['Restriction'].map(RestrictionUpdate)

In [140]:
TotalFlights2023.shape

(18303, 19)

**Add columns for L3 Description, L3 Node, & L4 Description**

In [137]:
deptid_key = pd.read_csv('/Users/ashley/deptid_key.csv')

In [138]:
TotalFlights2023_merged = TotalFlights2023.merge(deptid_key[['Deptid', 'OrgL3DivisionNode', 'OrgL3DivisionDesc', 'OrgL4DepartmentNode', 'OrgL4DepartmentDesc', 'OrgDesc']], on = 'Deptid', how = 'left')

In [141]:
TotalFlights2023_merged['RestrictionUpdate'].nunique()

3

In [142]:
TotalFlights2023_merged['RestrictionUpdate'].value_counts()

RestrictionUpdate
R              12571
U               5671
missingdata       61
Name: count, dtype: int64

In [143]:
TotalFlights2023_merged['RestrictionUpdate'].shape

(18303,)

In [144]:
TotalFlights2023_merged.shape

(18303, 24)

In [147]:
TotalFlights2023_merged.dtypes

which_agency             object
Origin                   object
Voucher ID                int64
Vendor Name              object
Invoice Date             object
Invoice ID (or DBID)     object
Payment Date             object
Invoice Amt             float64
Line Description         object
Paid Gross Amount       float64
Account                   int64
Fund                      int64
Deptid                    int64
Payment Method           object
Account Descr            object
Amount                  float64
ATMF                     object
Restriction              object
RestrictionUpdate        object
OrgL3DivisionNode        object
OrgL3DivisionDesc        object
OrgL4DepartmentNode      object
OrgL4DepartmentDesc      object
OrgDesc                  object
dtype: object

In [148]:
TotalFlights2023_merged['Deptid'].shape

(18303,)

In [149]:
TotalFlights2023_merged['Deptid'].isnull().sum()

0

In [150]:
TotalFlights2023_merged['OrgL3DivisionNode'].isnull().sum()

44

In [151]:
TotalFlights2023_merged['OrgL3DivisionDesc'].isnull().sum()

44

In [152]:
TotalFlights2023_merged['OrgL4DepartmentNode'].isnull().sum()

44

In [153]:
TotalFlights2023_merged['OrgL4DepartmentDesc'].isnull().sum()

44

In [154]:
TotalFlights2023_merged['OrgDesc'].isnull().sum()

44

In [155]:
TotalFlights2023_merged['OrgL3DivisionNode'].nunique()

40

In [156]:
TotalFlights2023_merged['OrgL3DivisionNode'].value_counts()

OrgL3DivisionNode
COENG    2615
VCRAC    2467
LS1SS    1423
VCUGA    1378
LS1PS    1227
SCLAW    1207
HAAS3    1017
LS1HU     926
LS1BS     666
CO1NR     657
COCHM     569
SC1PH     518
VCEI3     329
UCRLO     301
VCBAS     287
SCEDU     268
VCRMS     266
CENVD     260
GSCPP     248
DSDIV     185
SC1OP     159
SCSIM     139
SCHSW     111
ACCTL     109
UCLIB      97
SSALL      94
SCJOU      90
VCRAU      84
VR1GD      80
ATHLE      74
MU1FA      64
UNEX3      57
CALPF      52
CHANL      49
OT1VP      45
VPAPF      45
SAFP3      35
LS1UI      30
VCFIN      29
EVCP3       2
Name: count, dtype: int64

In [157]:
TotalFlights2023_merged.to_csv('TotalFlights2023_merged_10may24.csv', index = False)

**At this point, you've exported the Total Flights DataFrame with the line above (the .csv file)**

**Below, you may continue to create pivot tables based on the Total Flights DataFrame**

In [158]:
def unique_ids(series):
    return series.nunique()

In [159]:
index_cols_division_dept = ['OrgL3DivisionNode', 'OrgL4DepartmentDesc']

In [160]:
index_cols_division = ['OrgL3DivisionNode']

**After defining some indices, the next line creates a pivot table that returns flight totals from all units including their division and department**

In [161]:
pivot_all_units = TotalFlights2023_merged.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

**The next line creates a pivot table that returns total flights from all units and their associated divisions (excludes departments)**

In [162]:
pivot_all_units_divisions = TotalFlights2023_merged.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

In [163]:
pivot_all_units

RestrictionUpdate                                            R           \
ATMF                                             compedVisitor domestic   
OrgL3DivisionNode OrgL4DepartmentDesc                                     
ACCTL             Ctrl Ops Travel & Ent                    6.0     39.0   
ATHLE             Intercollegiate Athletics                NaN      NaN   
CALPF             Cal Performances                         NaN      1.0   
                  Student Musical Activities               NaN     19.0   
CENVD             City & Regional Planning                 NaN      1.0   
...                                                        ...      ...   
VPAPF             American Cultures                        NaN      NaN   
                  Berkeley Changemaker                     NaN      1.0   
                  Research Teaching and Learning           NaN     22.0   
                  Undergrad Edu Administration             NaN      1.0   
VR1GD             Graduate Division Ops                    NaN      9.0   

RestrictionUpdate                                                       \
ATMF                                             international unknown   
OrgL3DivisionNode OrgL4DepartmentDesc                                    
ACCTL             Ctrl Ops Travel & Ent                   11.0     NaN   
ATHLE             Intercollegiate Athletics                NaN     NaN   
CALPF             Cal Performances                         NaN     NaN   
                  Student Musical Activities               NaN     7.0   
CENVD             City & Regional Planning                 NaN     NaN   
...                                                        ...     ...   
VPAPF             American Cultures                        NaN     NaN   
                  Berkeley Changemaker                     NaN     NaN   
                  Research Teaching and Learning           1.0     NaN   
                  Undergrad Edu Administration             NaN     NaN   
VR1GD             Graduate Division Ops                    1.0     NaN   

RestrictionUpdate                                            U           \
ATMF                                             compedVisitor domestic   
OrgL3DivisionNode OrgL4DepartmentDesc                                     
ACCTL             Ctrl Ops Travel & Ent                    NaN     46.0   
ATHLE             Intercollegiate Athletics                NaN     66.0   
CALPF             Cal Performances                         NaN     22.0   
                  Student Musical Activities               NaN      NaN   
CENVD             City & Regional Planning                 NaN     14.0   
...                                                        ...      ...   
VPAPF             American Cultures                        NaN      3.0   
                  Berkeley Changemaker                     NaN      NaN   
                  Research Teaching and Learning           NaN     12.0   
                  Undergrad Edu Administration             NaN      3.0   
VR1GD             Graduate Division Ops                    NaN     59.0   

RestrictionUpdate                                                       \
ATMF                                             international unknown   
OrgL3DivisionNode OrgL4DepartmentDesc                                    
ACCTL             Ctrl Ops Travel & Ent                    7.0     NaN   
ATHLE             Intercollegiate Athletics                7.0     NaN   
CALPF             Cal Performances                         1.0     NaN   
                  Student Musical Activities               NaN     NaN   
CENVD             City & Regional Planning                11.0     NaN   
...                                                        ...     ...   
VPAPF             American Cultures                        NaN     NaN   
                  Berkeley Changemaker                     NaN     NaN   
                  Research Teaching and Learnin

In [164]:
pivot_all_units_divisions

RestrictionUpdate             R                                            U  \
ATMF              compedVisitor domestic international unknown compedVisitor   
OrgL3DivisionNode                                                              
ACCTL                       6.0     39.0          11.0     NaN           NaN   
ATHLE                       NaN      NaN           NaN     NaN           NaN   
CALPF                       NaN     20.0           NaN     7.0           NaN   
CENVD                       NaN    115.0          23.0     1.0           NaN   
CHANL                       NaN      5.0           NaN     NaN           NaN   
CO1NR                       2.0    389.0         120.0     1.0           NaN   
COCHM                      10.0    361.0          78.0     NaN           NaN   
COENG                      73.0   1305.0         693.0     2.0           NaN   
DSDIV                      22.0     78.0          27.0     NaN           NaN   
EVCP3                       NaN      NaN           NaN     NaN           NaN   
GSCPP                       1.0    149.0          31.0     NaN           NaN   
HAAS3                       NaN    425.0         156.0     NaN           NaN   
LS1BS                       3.0    303.0         129.0     NaN           NaN   
LS1HU                       NaN    387.0         197.0     2.0           NaN   
LS1PS                      55.0    602.0         260.0     NaN           NaN   
LS1SS                       NaN    533.0         243.0     NaN           NaN   
LS1UI                       NaN     10.0           NaN     NaN           NaN   
MU1FA                       NaN     38.0          12.0     3.0           NaN   
OT1VP                       NaN      8.0           NaN     NaN           NaN   
SAFP3                       NaN      NaN           NaN     NaN           NaN   
SC1OP                       NaN     98.0          12.0     NaN           NaN   
SC1PH                       1.0    278.0          97.0     NaN           NaN   
SCEDU                       NaN    149.0          34.0     NaN           NaN   
SCHSW                       NaN     63.0           8.0     NaN           NaN   
SCJOU                       NaN     47.0           7.0     NaN           NaN   
SCLAW                       NaN    328.0         103.0     1.0           NaN   
SCSIM                       1.0     35.0          46.0     NaN           NaN   
SSALL                       NaN      NaN           NaN     NaN           NaN   
UCLIB                       NaN     36.0           5.0     NaN           NaN   
UCRLO                       NaN     13.0           NaN     NaN           NaN   
UNEX3                       NaN      NaN           NaN     NaN           NaN   
VCBAS                       NaN    131.0           NaN     NaN           NaN   
VCEI3                       NaN    173.0          28.0     NaN           NaN   
VCFIN                       NaN      NaN           NaN     NaN           NaN   
VCRAC                      61.0   1427.0         587.0    15.0           NaN   
VCRAU                       NaN     17.0           8.0     NaN           NaN   
VCRMS                      52.0    100.0          31.0     NaN           NaN   
VCUGA                      24.0    346.0           2.0     1.0          14.0   
VPAPF                       NaN     24.0           1.0     NaN           NaN   
VR1GD                       NaN      9.0           1.0     NaN           NaN   

RestrictionUpdate                                missingdata                
ATMF              domestic international unknown    domestic international  
OrgL3DivisionNode                                                           
ACCTL                 46.0           7.0     NaN         NaN           NaN  
ATHLE                 66.0           7.0     NaN         NaN           NaN  
CALPF                 22.0           1.0     NaN         NaN           NaN  
CENVD                 72.0          26.0     NaN         NaN           NaN  
CHANL               

In [165]:
pivot_all_units_divisions.to_csv('pivot_all_units_divisions_2023_8may24.csv', index = True, index_label=['OrgL3DivisionNode'])

In [166]:
pivot_all_units.to_csv('pivot_all_units_2023_8may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc'])

**Create a pivot that shows all fund values associated with account 57239 - stagingArea**

In [48]:
#stagingArea_filter = TotalFlights2023_merged[TotalFlights2023_merged['Account'] == 57239]

In [60]:
#stagingArea_filter.shape

(6870, 26)

In [63]:
#pivot_stagingArea = pd.pivot_table(stagingArea_filter, values = 'Fund', columns = ['Account','which_agency'], aggfunc='count')

In [64]:
#pivot_stagingArea

Account      57239      
which_agency   BCD  UCTC
Fund          1702  5168

In [55]:
#pivot_stagingArea_count = pd.pivot_table(stagingArea_filter, values = 'Fund', columns = 'Account', aggfunc=list)

In [56]:
#pivot_stagingArea_count = pivot_stagingArea_count.apply(pd.Series.explode)

In [57]:
#unique_fund_stagingArea = pivot_stagingArea_count[57239].unique()

In [58]:
#unique_fund_stagingArea

array([69995], dtype=object)

**Defining more indices here**

In [2]:
optin_units = ['CO1NR', 'COCHM', 'COENG', 'COL1S', 'DSDIV', 'EVCP3', 'GSCPP', 'HAAS3', 'LS1PS', 'LS1UI', 'OT1VP', 'SCEDU', 'SCLAW', 'SSALL', 'UCRLO', 'UNEX3', 'VCBAS', 'VCEI3', 'VCRAC', 'VCRAU', 'VCRMS', 'VCUGA', 'VR1GD']

In [3]:
optin_units

['CO1NR',
 'COCHM',
 'COENG',
 'COL1S',
 'DSDIV',
 'EVCP3',
 'GSCPP',
 'HAAS3',
 'LS1PS',
 'LS1UI',
 'OT1VP',
 'SCEDU',
 'SCLAW',
 'SSALL',
 'UCRLO',
 'UNEX3',
 'VCBAS',
 'VCEI3',
 'VCRAC',
 'VCRAU',
 'VCRMS',
 'VCUGA',
 'VR1GD']

**Create a dataframe that only includes rows that are from our opt-in units**

In [168]:
filtered_optin = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(optin_units)]

**Here I created a dataframe that pulled all cells from the Opt-in filter that are missing Restriction Data. This is so that we can find their Fund Codes and assign them fees**

In [42]:
##RestrictionUpdate_missingdata = ['missingdata']

**These next few lines are because Kira requested we find all missing Fund codes in case more units join our program**

In [43]:
##missing_restriction_all = TotalFlights2023_merged[TotalFlights2023_merged['RestrictionUpdate'].isin(RestrictionUpdate_missingdata)]

In [44]:
##unique_fund_values_all = missing_restriction_all['Fund'].unique()


In [45]:
##unique_fund_values_all_df = pd.DataFrame({'Unique_Fund_Values_All':unique_fund_values_all})

In [46]:
##unique_fund_values_all_df.to_csv('all_missing_fund_values_2023.csv', index = False)

**Here are the missing Fund codes just for the Opt-in units**

In [42]:
##filtered_missing_restriction = filtered_optin[filtered_optin['RestrictionUpdate'].isin(RestrictionUpdate_missingdata)]

In [43]:
##filtered_missing_restriction.shape

(684, 26)

In [44]:
##filtered_missing_restriction.to_csv('optin_missing_restriction_data_2023.csv', index = False)

In [45]:
##unique_fund_values = filtered_missing_restriction['Fund'].unique()

In [46]:
##unique_fund_values_df = pd.DataFrame({'Unique_Fund_Values':unique_fund_values})

In [47]:
##unique_fund_values_df.to_csv('optin_missing_fund_values_2023.csv', index = False)

**Here, I created another dataframe that pulls all rows from the Opt-in filter that are missing 'ATMF' data. That way we can see which account codes we're missing information**

In [48]:
##ATMF_missingdata = ['unknown']

In [49]:
##filtered_missing_ATMF = filtered_optin[filtered_optin['ATMF'].isin(ATMF_missingdata)]

In [50]:
##filtered_missing_ATMF.shape

(274, 26)

In [51]:
##filtered_missing_ATMF['Account'].value_counts()

Account
56715    208
57239     66
Name: count, dtype: int64

In [52]:
##unique_account_values = filtered_missing_ATMF['Account'].unique()

In [53]:
##unique_account_values_df = pd.DataFrame({'Unique_Account_values':unique_account_values})

In [54]:
##unique_account_values_df.to_csv('optin_missing_account_values_2023.csv', index = False)

**Define a condition by calling the .nunique() attribute**

In [169]:
def unique_ids(series):
    return series.nunique()

**Creates a pivot table that only shows the unique 'Invoice ID (or DBID)' rows from the filtered opt-in dataframe**

**This is essentially the total restricted versus unrestricted flights per campus opt-in divison IF we can equate 1 flight with 1 'Invoice ID (or DBID)'**

In [1]:
filtered_optin.shape

NameError: name 'filtered_optin' is not defined

In [174]:
filtered_optin['OrgL3DivisionNode'].value_counts()

OrgL3DivisionNode
COENG    2615
VCRAC    2467
VCUGA    1378
LS1PS    1227
SCLAW    1207
HAAS3    1017
CO1NR     657
COCHM     569
VCEI3     329
UCRLO     301
VCBAS     287
SCEDU     268
VCRMS     266
GSCPP     248
DSDIV     185
SSALL      94
VCRAU      84
VR1GD      80
UNEX3      57
OT1VP      45
LS1UI      30
EVCP3       2
Name: count, dtype: int64

In [175]:
pivot_optin = filtered_optin.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division, columns = 'RestrictionUpdate', aggfunc=unique_ids)

In [176]:
pivot_optin

RestrictionUpdate,R,U,missingdata
OrgL3DivisionNode,,,
CO1NR,510.0,70.0,3.0
COCHM,449.0,59.0,NaN
COENG,2058.0,297.0,3.0
DSDIV,126.0,46.0,NaN
EVCP3,NaN,2.0,NaN
GSCPP,181.0,42.0,NaN
HAAS3,577.0,343.0,1.0
LS1PS,911.0,229.0,4.0
LS1UI,10.0,20.0,NaN


**This pivot table lists all opt-in units (including division & dept) and their associated R, U, and unknown flights**

In [117]:
#pivot_optin.to_csv('pivot_optin_RU_updatedagain.csv', index = True, index_label=['OrgL3DivisionNode'])

In [177]:
pivot_optin_2023 = filtered_optin.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

In [178]:
pivot_optin_2023

RestrictionUpdate                                          R           \
ATMF                                           compedVisitor domestic   
OrgL3DivisionNode OrgL4DepartmentDesc                                   
CO1NR             Agricultural Res Econ Pol              NaN     28.0   
                  CNR Office of the Dean                 NaN     35.0   
                  Energy & Resources Group ERG           NaN     26.0   
                  Environ Sci, Policy & Mgmt             2.0    163.0   
                  Nutritional Sci & Tox Dept             NaN     31.0   
...                                                      ...      ...   
VCUGA             Student Affairs Immed Off              NaN      NaN   
                  Student Affairs Services               NaN      NaN   
                  UG Admissions & Enrollment             NaN     71.0   
                  University Health Services             NaN      2.0   
VR1GD             Graduate Division Ops                  NaN      9.0   

RestrictionUpdate                                                     \
ATMF                                           international unknown   
OrgL3DivisionNode OrgL4DepartmentDesc                                  
CO1NR             Agricultural Res Econ Pol              6.0     NaN   
                  CNR Office of the Dean                 1.0     1.0   
                  Energy & Resources Group ERG          15.0     NaN   
                  Environ Sci, Policy & Mgmt            66.0     NaN   
                  Nutritional Sci & Tox Dept             7.0     NaN   
...                                                      ...     ...   
VCUGA             Student Affairs Immed Off              NaN     NaN   
                  Student Affairs Services               NaN     NaN   
                  UG Admissions & Enrollment             1.0     NaN   
                  University Health Services             NaN     NaN   
VR1GD             Graduate Division Ops                  1.0     NaN   

RestrictionUpdate                                          U           \
ATMF                                           compedVisitor domestic   
OrgL3DivisionNode OrgL4DepartmentDesc                                   
CO1NR             Agricultural Res Econ Pol              NaN      6.0   
                  CNR Office of the Dean                 NaN      NaN   
                  Energy & Resources Group ERG           NaN      9.0   
                  Environ Sci, Policy & Mgmt             NaN     33.0   
                  Nutritional Sci & Tox Dept             NaN      4.0   
...                                                      ...      ...   
VCUGA             Student Affairs Immed Off              NaN     50.0   
                  Student Affairs Services               NaN     10.0   
                  UG Admissions & Enrollment             NaN    204.0   
                  University Health Services             NaN     28.0   
VR1GD             Graduate Division Ops                  NaN     59.0   

RestrictionUpdate                                                     \
ATMF                                           international unknown   
OrgL3DivisionNode OrgL4DepartmentDesc                                  
CO1NR             Agricultural Res Econ Pol              2.0     NaN   
                  CNR Office of the Dean                 NaN     NaN   
                  Energy & Resources Group ERG           NaN     NaN   
                  Environ Sci, Policy & Mgmt             9.0     NaN   
                  Nutritional Sci & Tox Dept             NaN     NaN   
...                                                      ...     ...   
VCUGA             Student Affairs Immed Off              1.0     NaN   
                  Student Affairs Services               NaN     NaN   
                  UG Admissions & Enrollment             5.0     NaN   
                  University Health Services             6.0     NaN   
VR1GD    

In [179]:
pivot_optin_2023.to_csv('pivot_optin_updatedagain_2023.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc'])

**If you'd like to export a dataframe for each division, you can do so here**

In [184]:
CO1NR = ['CO1NR']

In [182]:
COCHM = ['COCHM']
COENG = ['COENG']
DSDIV = ['DSDIV']
GSCPP = ['GSCPP']
HAAS3 = ['HAAS3']
LS1PS = ['LS1PS']
LS1UI = ['LS1UI']
OT1VP = ['OT1VP']
SCEDU = ['SCEDU']
SCLAW = ['SCLAW']
SSALL = ['SSALL']
UCRLO = ['UCRLO']
UNEX3 = ['UNEX3']
VCBAS = ['VCBAS']
VCEI3 = ['VCEI3']
VCRAC = ['VCRAC']
VCRAU = ['VCRAU']
VCRMS = ['VCRMS']
VCUGA = ['VCUGA']
VR1GD = ['VR1GD']

In [185]:
CO1NR_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(CO1NR)]

In [186]:
CO1NR_2023_filter.shape

(657, 24)

In [188]:
COCHM_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(COCHM)]
COENG_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(COENG)]
DSDIV_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(DSDIV)]
GSCPP_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(GSCPP)]
HAAS3_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(HAAS3)]
LS1PS_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(LS1PS)]
LS1UI_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(LS1UI)]
OT1VP_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(OT1VP)]
SCEDU_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(SCEDU)]
SCLAW_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(SCLAW)]
SSALL_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(SSALL)]
UCRLO_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(UCRLO)]
UNEX3_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(UNEX3)]
VCBAS_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCBAS)]
VCEI3_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCEI3)]
VCRAC_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCRAC)]
VCRAU_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCRAU)]
VCRMS_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCRMS)]
VCUGA_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VCUGA)]
VR1GD_2023_filter = TotalFlights2023_merged[TotalFlights2023_merged['OrgL3DivisionNode'].isin(VR1GD)]

In [190]:
restricted = ['R']

In [191]:
CO1NR_restricted_2023 = CO1NR_2023_filter[CO1NR_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')

In [192]:
COCHM_restricted_2023 = COCHM_2023_filter[COCHM_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
COENG_restricted_2023 = COENG_2023_filter[COENG_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
DSDIV_restricted_2023 = DSDIV_2023_filter[DSDIV_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
GSCPP_restricted_2023 = GSCPP_2023_filter[GSCPP_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
HAAS3_restricted_2023 = HAAS3_2023_filter[HAAS3_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
LS1PS_restricted_2023 = LS1PS_2023_filter[LS1PS_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
LS1UI_restricted_2023 = LS1UI_2023_filter[LS1UI_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
OT1VP_restricted_2023 = OT1VP_2023_filter[OT1VP_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
SCEDU_restricted_2023 = SCEDU_2023_filter[SCEDU_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
SCLAW_restricted_2023 = SCLAW_2023_filter[SCLAW_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
SSALL_restricted_2023 = SSALL_2023_filter[SSALL_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
UCRLO_restricted_2023 = UCRLO_2023_filter[UCRLO_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
UNEX3_restricted_2023 = UNEX3_2023_filter[UNEX3_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCBAS_restricted_2023 = VCBAS_2023_filter[VCBAS_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCEI3_restricted_2023 = VCEI3_2023_filter[VCEI3_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCRAC_restricted_2023 = VCRAC_2023_filter[VCRAC_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCRAU_restricted_2023 = VCRAU_2023_filter[VCRAU_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCRMS_restricted_2023 = VCRMS_2023_filter[VCRMS_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VCUGA_restricted_2023 = VCUGA_2023_filter[VCUGA_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')
VR1GD_restricted_2023 = VR1GD_2023_filter[VR1GD_2023_filter['RestrictionUpdate'].isin(restricted)].drop_duplicates(subset='Invoice ID (or DBID)')


In [193]:
CO1NR_restricted_2023.shape

(510, 24)

**LEFT OFF HERE**

**Export a .csv with all rows from CO1NR**

In [154]:
CO1NR_restricted_2023.to_csv('CO1NR_restricted_2023_10may24.csv', index = False)

In [156]:
COCHM_restricted_2023.to_csv('COCHM_restricted_2023_10may24.csv', index = False)
DSDIV_restricted_2023.to_csv('DSDIV_restricted_2023_10may24.csv', index = False)
GSCPP_restricted_2023.to_csv('GSCPP_restricted_2023_10may24.csv', index = False)
HAAS3_restricted_2023.to_csv('HAAS3_restricted_2023_10may24.csv', index = False)
LS1PS_restricted_2023.to_csv('LS1PS_restricted_2023_10may24.csv', index = False)
LS1UI_restricted_2023.to_csv('LS1UI_restricted_2023_10may24.csv', index = False)
OT1VP_restricted_2023.to_csv('OT1VP_restricted_2023_10may24.csv', index = False)
SCEDU_restricted_2023.to_csv('SCEDU_restricted_2023_10may24.csv', index = False)
SCLAW_restricted_2023.to_csv('SCLAW_restricted_2023_10may24.csv', index = False)
SSALL_restricted_2023.to_csv('SSALL_restricted_2023_10may24.csv', index = False)
UCRLO_restricted_2023.to_csv('UCRLO_restricted_2023_10may24.csv', index = False) 
UNEX3_restricted_2023.to_csv('UNEX3_restricted_2023_10may24.csv', index = False)
VCBAS_restricted_2023.to_csv('VCBAS_restricted_2023_10may24.csv', index = False) 
VCEI3_restricted_2023.to_csv('VCEI3_restricted_2023_10may24.csv', index = False)
VCRAC_restricted_2023.to_csv('VCRAC_restricted_2023_10may24.csv', index = False)
VCRAU_restricted_2023.to_csv('VCRAU_restricted_2023_10may24.csv', index = False)
VCRMS_restricted_2023.to_csv('VCRMS_restricted_2023_10may24.csv', index = False)
VCUGA_restricted_2023.to_csv('VCUGA_restricted_2023_10may24.csv', index = False)
VR1GD_restricted_2023.to_csv('VR1GD_restricted_2023_10may24.csv', index = False)

In [194]:
COENG_restricted_2023.to_csv('COENG_restricted_2023_15july24.csv', index = False)

**Create a pivot table that shows total # of flights per Restriction (including Unrestricted & missing data)**

In [196]:
index_cols_division_dept_deptid = ['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid']

In [197]:
CO1NR_2023 = CO1NR_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

In [159]:
COCHM_2023 = COCHM_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
DSDIV_2023 = DSDIV_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
GSCPP_2023 = GSCPP_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
HAAS3_2023 = HAAS3_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
LS1PS_2023 = LS1PS_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
LS1UI_2023 = LS1UI_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
OT1VP_2023 = OT1VP_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
SCEDU_2023 = SCEDU_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
SCLAW_2023 = SCLAW_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
SSALL_2023 = SSALL_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
UCRLO_2023 = UCRLO_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
UNEX3_2023 = UNEX3_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCBAS_2023 = VCBAS_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCEI3_2023 = VCEI3_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCRAC_2023 = VCRAC_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCRAU_2023 = VCRAU_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCRMS_2023 = VCRMS_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VCUGA_2023 = VCUGA_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)
VR1GD_2023 = VR1GD_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

In [198]:
COENG_2023 = COENG_2023_filter.pivot_table(values = 'Invoice ID (or DBID)', index = index_cols_division_dept_deptid, columns = ('RestrictionUpdate', 'ATMF'), aggfunc=unique_ids)

In [199]:
COENG_2023

RestrictionUpdate                                           R           \
ATMF                                            compedVisitor domestic   
OrgL3DivisionNode OrgL4DepartmentDesc    Deptid                          
COENG             Bioengineering         11450            NaN      5.0   
                                         11452            NaN      4.0   
                                         11453            NaN      2.0   
                                         11458            NaN      NaN   
                                         11461            NaN      2.0   
...                                                       ...      ...   
                  Mechanical Engineering 11583            NaN     17.0   
                                         11584            NaN      1.0   
                  Nuclear Engineering    11595            NaN      2.0   
                                         11598            NaN     12.0   
                                         11599            NaN      NaN   

RestrictionUpdate                                                      \
ATMF                                            international unknown   
OrgL3DivisionNode OrgL4DepartmentDesc    Deptid                         
COENG             Bioengineering         11450            NaN     NaN   
                                         11452            2.0     NaN   
                                         11453            1.0     NaN   
                                         11458            NaN     NaN   
                                         11461            NaN     NaN   
...                                                       ...     ...   
                  Mechanical Engineering 11583           15.0     NaN   
                                         11584            NaN     NaN   
                  Nuclear Engineering    11595            NaN     NaN   
                                         11598            3.0     NaN   
                                         11599            NaN     NaN   

RestrictionUpdate                                      U                \
ATMF                                            domestic international   
OrgL3DivisionNode OrgL4DepartmentDesc    Deptid                          
COENG             Bioengineering         11450       NaN           NaN   
                                         11452       2.0           NaN   
                                         11453       NaN           NaN   
                                         11458       6.0           NaN   
                                         11461       NaN           NaN   
...                                                  ...           ...   
                  Mechanical Engineering 11583       9.0           2.0   
                                         11584       NaN           NaN   
                  Nuclear Engineering    11595       2.0           NaN   
                                         11598       3.0           8.0   
                                         11599       3.0           NaN   

RestrictionUpdate                               missingdata                
ATMF                                               domestic international  
OrgL3DivisionNode OrgL4DepartmentDesc    Deptid                            
COENG             Bioengineering         11450          NaN           NaN  
                                         11452          NaN           NaN  
                                         11453          NaN           NaN  
                                         11458          NaN           NaN  
                                         11461          NaN           NaN  
...                                                     ...           ...  
                  Mechanical Engineering 11583          NaN           NaN  
                                         11584          NaN           NaN  
                  Nuclear Engineering    11595          NaN        

**Export the pivot table. This only shows total amounts, not including full details - do this for every division**

In [161]:
CO1NR_2023.to_csv('CO1NR_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])

In [200]:
COENG_2023.to_csv('COENG_2023_15july24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])

In [162]:
COCHM_2023.to_csv('COCHM_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
DSDIV_2023.to_csv('DSDIV_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
GSCPP_2023.to_csv('GSCPP_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
HAAS3_2023.to_csv('HAAS3_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
LS1PS_2023.to_csv('LS1PS_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
LS1UI_2023.to_csv('LS1UI_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
OT1VP_2023.to_csv('OT1VP_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
SCEDU_2023.to_csv('SCEDU_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
SCLAW_2023.to_csv('SCLAW_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
SSALL_2023.to_csv('SSALL_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
UCRLO_2023.to_csv('UCRLO_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
UNEX3_2023.to_csv('UNEX3_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCBAS_2023.to_csv('VCBAS_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCEI3_2023.to_csv('VCEI3_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCRAC_2023.to_csv('VCRAC_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCRAU_2023.to_csv('VCRAU_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCRMS_2023.to_csv('VCRMS_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VCUGA_2023.to_csv('VCUGA_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])
VR1GD_2023.to_csv('VR1GD_2023_10may24.csv', index = True, index_label=['OrgL3DivisionNode', 'OrgL4DepartmentDesc', 'Deptid'])

##Major results per unit could be useful for reporting purposes 
Visual reports on their travel patterns 
month to month travel data
international vs domestic 
how much money is spent 
carbon emissions associated with their travel 
figure out if we can request more columns of data (mileage, destinations, etc.)
september 7th have a report ready for marina
send marina pie graph about emissions - we can do this with 2023 now 